# Ollama + OpenAI + Python

## 1. Angiv modelnavnet

Hvis du har hentet en anden model end "phi3:mini", skal du ændre værdien i cellen nedenfor. Den variabel vil blive brugt i koden gennem hele notebooken.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Opsæt Open AI-klienten

Normalt bruges OpenAI-klienten sammen med OpenAI.com eller Azure OpenAI til at interagere med store sprogmodeller.  
Dog kan den også bruges med Ollama, da Ollama tilbyder en OpenAI-kompatibel endpoint på "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generer en chatbesked

Nu kan vi bruge OpenAI SDK til at generere et svar til en samtale. Denne forespørgsel bør generere en haiku om katte:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Prompt engineering

Den første besked, der sendes til sprogmodellen, kaldes "systembeskeden" eller "systemprompten", og den fastsætter de overordnede instruktioner for modellen.  
Du kan give din egen tilpassede systemprompt for at guide en sprogmodel til at generere output på en anderledes måde.  
Rediger `SYSTEM_MESSAGE` nedenfor for at svare som din yndlingskarakter fra en berømt film/TV-serie, eller få inspiration til andre systemprompter fra [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Når du har tilpasset systembeskeden, skal du give det første bruger-spørgsmål i `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Eksempler med få skud

En anden måde at vejlede en sprogmodel på er at give "få skud", en række eksempler på spørgsmål/svar, der viser, hvordan den skal reagere.

Eksemplet nedenfor forsøger at få en sprogmodel til at opføre sig som en undervisningsassistent ved at give nogle eksempler på spørgsmål og svar, som en TA kunne give, og derefter stille modellen et spørgsmål, som en studerende kunne stille.

Prøv det først, og derefter kan du ændre `SYSTEM_MESSAGE`, `EXAMPLES` og `USER_MESSAGE` til et nyt scenarie.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Retrieval Augmented Generation

RAG (Retrieval Augmented Generation) er en teknik til at få en sprogmodel til at besvare spørgsmål præcist inden for et specifikt område, ved først at hente relevant information fra en videnskilde og derefter generere et svar baseret på denne information.

Vi har leveret en lokal CSV-fil med data om hybridbiler. Koden nedenfor læser CSV-filen, søger efter match med brugerens spørgsmål og genererer derefter et svar baseret på den fundne information. Bemærk, at dette vil tage længere tid end nogen af de tidligere eksempler, da det sender mere data til modellen. Hvis du bemærker, at svaret stadig ikke er baseret på dataene, kan du prøve system engineering eller andre modeller. Generelt er RAG mere effektivt med enten større modeller eller med finjusterede versioner af SLM'er.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
